In [1]:
import numpy as np
import pandas as pd
%load_ext autoreload

%autoreload 2
# import sys
# sys.path.insert(0, ".")

import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv('data/train_data_for_students.tsv', sep='\t', header=None)
df.columns = ['id', 'handle', 'tweet', 'date', 'device']
print(f'{df.shape=}')
df.head()


df.shape=(2682, 5)


,id,handle,tweet,date,device
0,845974102619906048,realDonaldTrump,Democrats are smiling in D.C. that the Freedom...,2017-03-26 15:21:58,iphone
1,846166053663191040,realDonaldTrump,General Kelly is doing a great job at the bord...,2017-03-27 04:04:42,iphone
2,835814988686233601,realDonaldTrump,"The race for DNC Chairman was, of course, tota...",2017-02-26 13:33:16,android
3,835817351178301440,realDonaldTrump,For first time the failing @nytimes will take ...,2017-02-26 13:42:39,android
4,835916511944523777,realDonaldTrump,"Russia talk is FAKE NEWS put out by the Dems, ...",2017-02-26 20:16:41,android


In [3]:
df.device.value_counts()


android                                                                                1683
iphone                                                                                  755
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                      201
<a href="http://www.twitter.com" rel="nofollow">Twitter for BlackBerry</a>               13
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>       9
<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>         4
<a href="http://instagram.com" rel="nofollow">Instagram</a>                               3
<a href="https://periscope.tv" rel="nofollow">Periscope.TV</a>                            2
<a href="http://www.facebook.com/twitter" rel="nofollow">Facebook</a>                     1
Name: device, dtype: int64

There is issue with the label column. There are values that not iphone / android. WTF?

In [4]:
df = df[(df.device == 'iphone') | (df.device == 'android')]
df.device.value_counts()

android    1683
iphone      755
Name: device, dtype: int64

In [5]:
# add numric label column
# android = 1
# iphone = 0

df['label'] = 0
df.loc[df['device'] == 'android', 'label'] = 1

In [6]:
from data_processing import create_folds

# Using StratifiedKfold since the label is not that balanced

NUMBER_OF_FOLDS = 3

df = create_folds(df, label_name='device', num_folds=NUMBER_OF_FOLDS, seed=11)

input df shape : (2438, 6)
Number of folds: 3, total samples (after removing NaN): 2438
fold: 0, num samples: 813
fold: 1, num samples: 813
fold: 2, num samples: 812


In [7]:
from data_processing import preprocess

# remove urls from tweets snice all hte urls with tweeter shortener

df.tweet = df.tweet.apply(preprocess)

In [8]:
5e-3

0.005

In [9]:
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from model import Trainer
from tqdm.auto import tqdm

lr_optios = [5e-5, 5e-4]
weight_decay_optios = [5e-3, 1e-2, 2e-2]

preds = df[['id', 'label', 'fold']]
total = len(lr_optios)*len(weight_decay_optios)*NUMBER_OF_FOLDS
with tqdm(total=total, desc = "CV steps"):
    for lr in lr_optios:
        for wd in weight_decay_optios:
            df.loc[:, 'y_pred'] = -1
            for fold in range(NUMBER_OF_FOLDS):
                output_name = f'lr-{lr}_wd-{wd}'
                tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
                model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
                optimizer = AdamW(model.parameters(), lr=lr, weight_decay=wd)

                trainer = Trainer(data=df,
                                tokenizer=tokenizer,
                                model=model,
                                optimizer=optimizer,
                                fold = fold,
                                early_stopping=2,
                                batch_size=8,
                                num_epochs=6,
                                output_name=output_name)


            
                trainer.train()
                trainer.final_eval()
                df.loc[:, 'y_pred'] = trainer.data.loc[:, 'y_pred']
            preds.loc[:, output_name] = df.loc[:, 'y_pred']



CV steps:   0%|          | 0/18 [00:00<?, ?it/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to C:\Users\itama\.cache\huggingface\datasets\csv\default-44ce9edaa6d9ad28\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1625 [00:00<?, ?ex/s]

  0%|          | 0/813 [00:00<?, ?ex/s]

EPOCHS:   0%|          | 0/6 [00:00<?, ?it/s]

train epoch:   0%|          | 0/204 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
preds